## Input Parameters

In [1]:
# This would show NLCD 41 points for all MGRS grid zones
point_nlcd_keep_list = [41]
point_mgrs_keep_list = []

# The minimum number of months in the target year
min_month_count = 3

# Overwrite the summary stats file everytime this block is run
output_txt = 'summary_stats_nlcd41.txt'
output_f = open(output_txt, 'w')
output_f.write('MGRS: '+ ', '.join(point_mgrs_keep_list) + '\n')
output_f.write('NLCD: '+ ', '.join(map(str, point_nlcd_keep_list)) + '\n\n')
output_f.close()

## Other Parameters

In [2]:
data_folder = 'data'
points_folder = 'points'

points_csv = 'gap_fill_test_points.csv'

# Exclude 2016 from the statistics since there is not a full prior year to interpolate from
# Including 2024 even though 2025 is not complete
stats_years = list(range(2017, 2025))

# Points were only built for NLCD 2024
nlcd_years = [2024]

# Include all MGRS grid zones except 12R and 16U since they are too small
mgrs_zones = [
    '10S', '10T', '10U', '11S', '11T', '11U', '12S', '12T', '12U', 
    '13R', '13S', '13T', '13U', '14R', '14S', '14T', '14U', '15R', '15S', '15T', '15U', 
    '16R', '16S', '16T', '17R', '17S', '17T', '18S', '18T', '19T'
    # '12R', '16U'
]

months = list(range(1, 13))

# TODO: Add support for setting a minimum number of months in the year
#   and minimum number of months in the growing season
# min_month_count = 6
# min_gs_month_count = 3


## Python Imports

In [3]:
from datetime import datetime
import math
import os
import pprint
import random

#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
import sklearn.metrics
import sklearn.linear_model

from whittaker_eilers import WhittakerSmoother

## Read the point CSV files

In [4]:
# Building a single points dataframe and CSV from the MGRS grid zone points CSV files
overwrite_flag = False

# Read the separate points CSV files into a single dataframe
points_df_list = [
    pd.read_csv(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'), index_col=None, header=0)
    for nlcd_year in nlcd_years
    for mgrs_zone in mgrs_zones
    if os.path.isfile(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'))
]
points_df = pd.concat(points_df_list, axis=0, ignore_index=True)
print(f'Points: {len(points_df.index)}')

# The mgrs_zone value will eventually be added to the csv files
points_df['mgrs_zone'] = points_df['mgrs_tile'].str.slice(0, 3)

# Add a unique index to the points dataframe
points_df['index_group'] = points_df.groupby(['mgrs_tile', 'nlcd']).cumcount()
points_df['point_id'] = (
    points_df["mgrs_tile"].str.upper() + '_' +
    'nlcd' + points_df["nlcd"].astype(str).str.zfill(2) + '_' +
    points_df["index_group"].astype(str).str.zfill(2)
)
del points_df['index_group']

# Round the lat and lon to 8 decimal places (probably should be 6)
points_df['latitude'] = round(points_df['latitude'], 8)
points_df['longitude'] = round(points_df['longitude'], 8)

# # Write to CSV
# if not os.path.isfile(points_csv) or overwrite_flag:
#     print('Writing points csv')
#     points_df.to_csv(points_csv, index=False)


Points: 17084


## Read the data CSV files

This block may take a little while to execute

In [5]:
# Read the CSV files into separate dataframes for each point
print('Reading mgrs data csv files')
data_df_dict = {}
for mgrs_zone in mgrs_zones:
    # print(mgrs_zone)
    if not os.path.isfile(os.path.join(data_folder, f'data_{mgrs_zone}.csv')):
        continue
        
    mgrs_df = pd.read_csv(os.path.join(data_folder, f'data_{mgrs_zone}.csv'), index_col=None, header=0)

    # Set MGRS value to upper case 
    # (at some point change this in all the data CSV files)
    mgrs_df['mgrs_tile'] = mgrs_df['mgrs_tile'].str.upper()
    mgrs_df['mgrs_zone'] = mgrs_df['mgrs_zone'].str.upper()
    
    # Compute the ET fraction
    mgrs_df['etf'] = mgrs_df['et'] / mgrs_df['eto']
    
    # Get the month for computing climos
    mgrs_df['date'] = pd.to_datetime(mgrs_df['date'])
    mgrs_df['year'] = mgrs_df['date'].dt.year
    mgrs_df['month'] = mgrs_df['date'].dt.month
    
    # Confirm that specific NLCD categories are not included
    # TODO: This probably isn't needed and switch to a check instead of masking
    for nlcd_skip in [11, 12, 21, 22, 23]:
        mgrs_df = mgrs_df[mgrs_df['nlcd'] != nlcd_skip]

    # Save dataframe for each point
    for point_id in mgrs_df['point_id'].unique():
        site_df = mgrs_df.loc[mgrs_df['point_id']==point_id].copy()
        site_df.set_index('date', drop=True, inplace=True)
        site_df.sort_index(inplace=True)
        data_df_dict[point_id] = site_df
        del site_df
    del mgrs_df

print('\nDone')

Reading mgrs data csv files

Done


## Compute the ETf climos

In [6]:
# Compute the maximum ETf per site
# Assuming it is okay to make this for the full period of record
print('\nComputing maximum ETf')
etf_max_dict = {
    point_id: data_df_dict[point_id].agg(etf=('etf', 'max'))['etf'].to_dict()['etf']
    for point_id in data_df_dict.keys()
}

# Compute climos for each site
# Only keep the climo value if there are at least "n" years of data
print('\nComputing monthly climatologies')
month_climo_count_min = 2
month_climo_dict = {}
for point_id in data_df_dict.keys():
    month_climo = data_df_dict[point_id].groupby(['month']).agg(
        etf=('etf', 'mean'), 
        etf_median=('etf', 'median'), 
        count=('etf', 'count'), 
        et=('et', 'mean'), 
        eto=('eto', 'mean'),
    )
    month_climo_count_mask = month_climo['count'] < month_climo_count_min
    month_climo.loc[month_climo_count_mask, ['etf', 'etf_median', 'et']] = np.nan
    month_climo_dict[point_id] = month_climo
    del month_climo, month_climo_count_mask

# # Compute climos for each target year that have the target year values excluded
# # CGM - This might be worth testing more but doesn't seem worth the time to generate
# print('\nComputing monthly climatologies for target years')
# month_climo_dict = {}
# for point_id in data_df_dict.keys():
#     if point_id not in month_climo_dict.keys():
#         month_climo_dict[point_id] = {}
#     for year in stats_years:
#         month_climo_dict[point_id][year] = (
#             data_df_dict[point_id][data_df_dict[point_id].year != year]
#             .groupby(['month'])
#             .agg(
#                 etf=('etf', 'mean'), 
#                 etf_median=('etf', 'median'), 
#                 count=('etf', 'count'), 
#                 et=('et', 'mean'), 
#                 eto=('eto', 'mean'),
#             )
#         )

# TODO: Join the climo values to the data dictionaries
#   It might be faster to join the climo data here instead of looking up in the function

print('\nDone')


Computing maximum ETf

Computing monthly climatologies

Done


## Functions for computing filled values and summary statistics

In [7]:
def generate_windows(
        point_id_list, 
        months=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
        years=[2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], 
        exclude_months_without_climo=True
):
    """Generate Window Dataframes"""
    for i, point_id in enumerate(point_id_list):
        # Assume the data df dictionary exists in the global scope
        site_df = data_df_dict[point_id]
        
        for year in years:
            # Pull a three year window for each target year so that there are images to interpolate and fill from
            window_df = site_df[(site_df.index.year >= (year-1)) & (site_df.index.year <= (year+1))].copy()

            # If excluding months without climos, set them to NaN here
            # TODO: Test out adding the climo values to the window_df here 
            #   instead of in the compute_filled_values() function below
            if exclude_months_without_climo:
                merge_df = pd.merge(window_df[['month']], month_climo_dict[point_id]['etf'], how="left", on="month")
                climo_nan_mask = merge_df['etf'].isna().values
                window_df.loc[climo_nan_mask, ['etf', 'et', 'count']] = np.nan

            year_mask = window_df.index.year==year
            year_month_mask = year_mask & window_df.index.month.isin(months)
            
            if window_df.loc[year_mask, 'etf'].count() < min_month_count:
                # Check if there are enough months in the target year
                # print(f'{point_id} - {i} - {year} - not enough unmasked months, skipping')
                continue
            elif window_df.loc[year_month_mask, 'etf'].isna().all():
                # Check that there are target months with data in the 
                # print(f'{point_id} - {i} - {year} - no unmasked months in year/months, skipping')
                continue
            elif (window_df.loc[(window_df.index.year==year-1), 'etf'].isna().all() or 
                  window_df.loc[(window_df.index.year==year+1), 'etf'].isna().all()):
                # Check if there is data in the prev/next year to interpolate from
                # print(f'{point_id} - {i} - {year} - no unmasked months in next/prev year, skipping')
                continue

            yield point_id, year, window_df, year_month_mask


comparison_cols = [
    'interpolate', 'climo_mean', 'climo_median',
    'conor',
    'interp_clim_a', 'interp_clim_b', 'interp_clim_c',
    'whit_a_0p50', 'whit_a_0p20', 'whit_a_0p10', 'whit_a_0p05', 'whit_a_0p01', 
]


def compute_filled_values(window_df, tgt_indices, point_id):
    """"""
    # Get a copy of the target value before clearing
    original_etf = window_df.loc[tgt_indices, 'etf'].values
        
    # Set the target row values to NaN
    window_df.loc[tgt_indices, ('etf', 'et', 'count')] = np.nan
    
    # Setup the Whittaker Smoothing for the full dataframe outside of the index loop
    # The smoothing function needs all nans filled with a value
    # The fill value is not important as long as the weight value is set to 0
    window_df['temp'] = window_df['etf'].copy()
    window_df.loc[np.isnan(window_df['temp']), 'temp'] = -1
    etf = window_df['temp'].values

    # TODO: Make sure weights are set to 0 for all temp==-1 rows
    #   This might be happening already with the .fillna(0) call but double check
    #   Right now the code is assuming count is NaN if etf is NaN
    
    # Default weights with 1 for data and 0 for missing values
    weight_a = window_df['count'].clip(lower=1, upper=1).fillna(0)
    if not any(weight_a):
        print(f'{point_id} - {i} - {year} - all weights 0, skipping')
        return []
    # CGM - I tested out building the smoother once and then updating lambda,
    #   but it didn't seem any faster
    whit_a_0p50 = WhittakerSmoother(lmbda=0.5, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p20 = WhittakerSmoother(lmbda=0.2, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p10 = WhittakerSmoother(lmbda=0.1, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p05 = WhittakerSmoother(lmbda=0.05, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p01 = WhittakerSmoother(lmbda=0.01, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)

    # CGM - I was testing out trying different weights but it didn't seem to change the values at all
    # # Compute weights based on the the scene count value
    # # Set count 0 images to a weight of 0
    # weight = window_df['count'].clip(lower=0, upper=1).fillna(0)

    # # Compute weights based on the the scene count value
    # # Set counts of 0 to a weight of 0.5 and all other to 1    
    # weight = window_df['count'].add(1).clip(upper=2).divide(2).fillna(0)

    # # Compute weights based on the scene count value
    # # Set count weights as: 0 -> 0, 1 -> 0.5, 2+ -> 1
    # weight = window_df['count'].fillna(0).clip(upper=2).divide(2)
    
    # Process each target index separately
    values = []
    for i, (tgt_index, tgt_i) in enumerate(zip(tgt_indices, window_df.index.get_indexer(tgt_indices))):

        interp_value = window_df['etf'].interpolate(method='linear').loc[tgt_index]

        # Climos for all years
        climo_mean = month_climo_dict[point_id].loc[tgt_index.month, 'etf']
        climo_count = month_climo_dict[point_id].loc[tgt_index.month, 'count']
        climo_median = month_climo_dict[point_id].loc[tgt_index.month, 'etf_median']
        # # Climos with the target year excluded (not sure if this matters)
        # climo_mean = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf']
        # climo_count = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'count']
        # climo_median = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf_median']

        # Compute various combinations of averaging the climo and interpolate values
        # Simple mean
        interp_clim_a = (climo_mean + interp_value) / 2
        # Simple mean with the median climo
        interp_clim_c = (climo_median + interp_value) / 2
        # Weight the climo based on the number of months in the climo?
        climo_months = 10
        interp_clim_b = (climo_mean * climo_count + interp_value * climo_months) / (climo_count + climo_months)

        # Conor's Approach
        # There is probably an easier way, but splitting the dataframe at the target index seemed to work pretty well
        window_prev_df = window_df.iloc[:tgt_i]
        window_next_df = window_df.iloc[tgt_i+1:]
        prev_index = window_prev_df['etf'].last_valid_index()
        next_index = window_next_df['etf'].first_valid_index()
        prev_i = window_df.index.get_loc(prev_index)
        next_i = window_df.index.get_loc(next_index)
        w_prev = 0.5 * math.exp(1 - (tgt_i - prev_i))
        w_next = 0.5 * math.exp(1 - (next_i - tgt_i))
        value_prev = window_df['etf'].iloc[prev_i]
        value_next = window_df['etf'].iloc[next_i]
        climo_prev = month_climo_dict[point_id].loc[prev_index.month, 'etf']
        climo_next = month_climo_dict[point_id].loc[next_index.month, 'etf']
        conor = w_prev * (value_prev - climo_prev) + w_next * (value_next - climo_next) + climo_mean

        values.append({
            'index': tgt_index,
            'point_id': point_id,
            'mgrs': point_id.split('_')[0],
            'nlcd': int(point_id.split('_')[1][4:6]),
            'original': original_etf[i],
            # Filled values
            'interpolate': interp_value,
            'climo_mean': climo_mean,
            'climo_median': climo_median,
            'conor': conor,
            'interp_clim_a': interp_clim_a,
            'interp_clim_b': interp_clim_b,
            'interp_clim_c': interp_clim_c,
            'whit_a_0p50': min(max(whit_a_0p50[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p20': min(max(whit_a_0p20[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p10': min(max(whit_a_0p10[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p05': min(max(whit_a_0p05[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p01': min(max(whit_a_0p01[tgt_i], 0), etf_max_dict[point_id]),
            
        })

    return values


def comparison_stats(df, x_col='original', y_cols=[], title='', print_flag=True, write_flag=True):
    """"""
    output = [title]

    # TODO: Build the format strings based on the number of parameters instead of hardcoding
    output.append('  {:>16s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s}'.format(
        'method', 'rmse', 'mae', 'mbe', 'm', 'b', 'r2', 'n'
    ))
    for y_col in y_cols:
        # Remove any NaN rows before computing statistics
        stat_df = df[df[y_col].notna()]
        model = sklearn.linear_model.LinearRegression()
        model.fit(stat_df[[x_col]], stat_df[y_col])

        output.append('  {:>16s} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8d}'.format(
            y_col,
            sklearn.metrics.root_mean_squared_error(stat_df[x_col], stat_df[y_col]),
            sklearn.metrics.mean_absolute_error(stat_df[x_col], stat_df[y_col]),
            np.mean(stat_df[y_col] - stat_df[x_col]),
            # np.mean(stat_df[x_col] - stat_df[y_col]),
            # sklearn.metrics.r2_score(stat_df[x_col], stat_df[y_col]),
            model.coef_[0],
            model.intercept_, 
            model.score(stat_df[[x_col]], stat_df[y_col]),
            # This count doesn't seem to change even when there are NaN values in the dataframe
            stat_df[y_col].count(),
        ))

    if print_flag:
        print('\n'.join(output))
    if write_flag:
        with open(output_txt, 'a') as output_f:
            output_f.write('\n'.join(output+ ['\n']))


## Filter to the target MGRS and NLCD classes

In [8]:
# Filter the points list to the target NLCD classes and MGRS grid zones
point_id_list = list(data_df_dict.keys())
if point_nlcd_keep_list:
    point_id_list = [p for p in point_id_list if int(p.split('_')[1][4:6]) in point_nlcd_keep_list]
if point_mgrs_keep_list:
    point_id_list = [p for p in point_id_list if p.split('_')[0][0:3] in point_mgrs_keep_list]

print(f'Points: {len(point_id_list)}')

Points: 1510


## Randomly drop one datapoint in each year

In [9]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop one datapoint in each year',
    print_flag=True, write_flag=True
)

Randomly drop one datapoint in each year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1231   0.0924   0.0018   0.7706   0.1692   0.7971    12080
        climo_mean   0.1054   0.0795  -0.0010   0.8536   0.1059   0.8507    12080
      climo_median   0.1083   0.0768  -0.0012   0.8686   0.0947   0.8429    12080
             conor   0.0982   0.0740  -0.0011   0.9004   0.0716   0.8712    12080
     interp_clim_a   0.1010   0.0765   0.0004   0.8121   0.1376   0.8660    12080
     interp_clim_b   0.1020   0.0770   0.0010   0.8097   0.1399   0.8632    12080
     interp_clim_c   0.1015   0.0756   0.0003   0.8196   0.1320   0.8635    12080
       whit_a_0p50   0.1226   0.0920   0.0006   0.8084   0.1404   0.7979    12080
       whit_a_0p20   0.1214   0.0904   0.0001   0.8392   0.1174   0.8034    12080
       whit_a_0p10   0.1219   0.0905  -0.0002   0.8534   0.1068   0.8033    12080
       whit_a_0p05   0.1228   0.0912  -0.0004   0.8622   

## Randomly drop a single datapoint from the "growing" season (Apr-Sept)

In [10]:
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
        
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint from the "growing" season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint from the "growing" season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1054   0.0809  -0.0199   0.7992   0.1520   0.8098    12080
        climo_mean   0.0937   0.0711   0.0012   0.8424   0.1362   0.8442    12080
      climo_median   0.0965   0.0691   0.0043   0.8507   0.1322   0.8354    12080
             conor   0.0826   0.0634   0.0003   0.9062   0.0807   0.8795    12080
     interp_clim_a   0.0866   0.0669  -0.0093   0.8208   0.1441   0.8713    12080
     interp_clim_b   0.0869   0.0670  -0.0093   0.8200   0.1449   0.8704    12080
     interp_clim_c   0.0871   0.0661  -0.0078   0.8250   0.1421   0.8687    12080
       whit_a_0p50   0.1053   0.0807  -0.0103   0.8173   0.1462   0.8051    12080
       whit_a_0p20   0.1034   0.0788  -0.0056   0.8439   0.1281   0.8121    12080
       whit_a_0p10   0.1034   0.0786  -0.0038   0.8568   0.1189   0.8129    12080
       whit_a_0p05   0.1040 

## Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)

In [11]:
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1497   0.1135   0.0401   0.6784   0.2197   0.5985    12017
        climo_mean   0.1216   0.0922  -0.0021   0.7017   0.1646   0.7071    12017
      climo_median   0.1257   0.0891  -0.0067   0.7138   0.1531   0.6887    12017
             conor   0.1174   0.0893  -0.0024   0.7773   0.1220   0.7301    12017
     interp_clim_a   0.1203   0.0921   0.0190   0.6900   0.1922   0.7217    12017
     interp_clim_b   0.1226   0.0935   0.0206   0.6862   0.1958   0.7115    12017
     interp_clim_c   0.1209   0.0912   0.0167   0.6961   0.1864   0.7162    12017
       whit_a_0p50   0.1469   0.1115   0.0232   0.7004   0.1905   0.5997    12017
       whit_a_0p20   0.1459   0.1101   0.0142   0.7240   0.1683   0.6055    12017
       whit_a_0p10   0.1469   0.1106   0.0103   0.7359   0.1578   0.6037    12017
       whit_a_0p05   0.14

## Randomly drop a datapoint from the winter (Dec-Feb)

In [12]:
months = [12, 1, 2]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))

output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a datapoint from the winter (Dec-Feb)',
    print_flag=True, write_flag=True
)

Randomly drop a datapoint from the winter (Dec-Feb)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1432   0.1076   0.0372   0.6601   0.2035   0.5666     9515
        climo_mean   0.1246   0.0945   0.0029   0.6384   0.1798   0.6356     9515
      climo_median   0.1285   0.0901  -0.0033   0.6476   0.1691   0.6144     9515
             conor   0.1194   0.0901   0.0027   0.7376   0.1311   0.6720     9515
     interp_clim_a   0.1205   0.0913   0.0200   0.6493   0.1916   0.6690     9515
     interp_clim_b   0.1242   0.0936   0.0226   0.6437   0.1970   0.6500     9515
     interp_clim_c   0.1212   0.0900   0.0169   0.6538   0.1863   0.6620     9515
       whit_a_0p50   0.1417   0.1061   0.0119   0.6742   0.1713   0.5565     9515
       whit_a_0p20   0.1427   0.1060   0.0039   0.6931   0.1540   0.5560     9515
       whit_a_0p10   0.1445   0.1070   0.0014   0.7036   0.1464   0.5517     9515
       whit_a_0p05   0.1465   0.1084   0.0004 

## Randomly drop a single datapoint that is next to an existing missing data point 

In [13]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # Skip the year if there are no NaN values
    if not window_df.loc[year_month_mask, 'etf'].isna().any():
        # print(f'{point_id} - {i} - {year} - no missing data points, skipping')
        continue
    
    # For the target year, pick a random month that is next to a missing/masked month but has data
    nan_mask = window_df['etf'].isna()
    tgt_mask = (
        (nan_mask | nan_mask.shift(1) | nan_mask.shift(-1))
        & window_df['etf'].notna() & year_month_mask
    )
    if not tgt_mask.any():
        print(f'{point_id} - {i} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint that is next to an existing missing data point',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint that is next to an existing missing data point
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1874   0.1464   0.0613   0.6042   0.2971   0.5238     6468
        climo_mean   0.1386   0.1055  -0.0096   0.6942   0.1726   0.7023     6468
      climo_median   0.1448   0.1024  -0.0148   0.7036   0.1618   0.6777     6468
             conor   0.1322   0.1008  -0.0088   0.7351   0.1490   0.7289     6468
     interp_clim_a   0.1448   0.1126   0.0258   0.6492   0.2349   0.6857     6468
     interp_clim_b   0.1490   0.1157   0.0298   0.6404   0.2440   0.6691     6468
     interp_clim_c   0.1461   0.1120   0.0232   0.6539   0.2294   0.6765     6468
       whit_a_0p50   0.1828   0.1424   0.0297   0.6780   0.2215   0.5326     6468
       whit_a_0p20   0.1831   0.1422   0.0165   0.7111   0.1886   0.5378     6468
       whit_a_0p10   0.1853   0.1438   0.0107   0.7266   0.1736   0.5353     6468
       whit_a_0p05

## Randomly drop a two month gap during the year

But only check the filled value in one month of the gap

In [14]:
dropped_months = 2
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))

    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1529   0.1164   0.0094   0.6709   0.2465   0.6868    12080
        climo_mean   0.1070   0.0807  -0.0003   0.8449   0.1115   0.8458    12080
      climo_median   0.1103   0.0782  -0.0005   0.8584   0.1015   0.8367    12080
             conor   0.1011   0.0761  -0.0001   0.8761   0.0891   0.8627    12080
     interp_clim_a   0.1132   0.0862   0.0045   0.7579   0.1790   0.8348    12080
     interp_clim_b   0.1155   0.0875   0.0059   0.7523   0.1844   0.8279    12080
     interp_clim_c   0.1135   0.0856   0.0044   0.7646   0.1740   0.8324    12080
       whit_a_0p50   0.1473   0.1113   0.0081   0.7492   0.1888   0.7110    12080
       whit_a_0p20   0.1463   0.1103   0.0071   0.7864   0.1610   0.7190    12080
       whit_a_0p10   0.1474   0.1110   0.0066   0.8034   0.1482   0.7182    12080
       whit_a_0p05   0.1491   0.1122   0.0062   0.81

## Randomly drop a two month gap during the growing season (Apr-Sept)

In [15]:
dropped_months = 2
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1313   0.1020  -0.0437   0.7447   0.1751   0.7249    12080
        climo_mean   0.0924   0.0708  -0.0005   0.8457   0.1317   0.8466    12080
      climo_median   0.0946   0.0686   0.0024   0.8563   0.1256   0.8396    12080
             conor   0.0844   0.0647  -0.0008   0.8832   0.0994   0.8721    12080
     interp_clim_a   0.0962   0.0748  -0.0221   0.7952   0.1534   0.8455    12080
     interp_clim_b   0.0966   0.0750  -0.0221   0.7943   0.1542   0.8441    12080
     interp_clim_c   0.0960   0.0739  -0.0206   0.8005   0.1504   0.8443    12080
       whit_a_0p50   0.1266   0.0972  -0.0223   0.7878   0.1596   0.7262    12080
       whit_a_0p20   0.1250   0.0955  -0.0161   0.8175   0.1403   0.7335    12080
       whit_a_0p10   0.1257   0.0958  -0.0137   0.8314   0.1309   0.7328    12080
       whit_a_0p05   0.1269   0

## Randomly drop a two month gap during the non-growing season (Oct-Mar)

In [16]:
dropped_months = 2
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1785   0.1377   0.0764   0.6296   0.2805   0.5068    12017
        climo_mean   0.1225   0.0933  -0.0026   0.6945   0.1657   0.6987    12017
      climo_median   0.1265   0.0901  -0.0072   0.7073   0.1540   0.6807    12017
             conor   0.1184   0.0904  -0.0028   0.7375   0.1418   0.7190    12017
     interp_clim_a   0.1316   0.1020   0.0369   0.6621   0.2231   0.6797    12017
     interp_clim_b   0.1359   0.1048   0.0403   0.6540   0.2310   0.6614    12017
     interp_clim_c   0.1317   0.1009   0.0346   0.6685   0.2173   0.6757    12017
       whit_a_0p50   0.1702   0.1306   0.0448   0.6704   0.2265   0.5094    12017
       whit_a_0p20   0.1700   0.1300   0.0333   0.6969   0.2003   0.5098    12017
       whit_a_0p10   0.1721   0.1313   0.0283   0.7100   0.1881   0.5047    12017
       whit_a_0p05   0.1745 

## Randomly drop a three month gap during the year

In [17]:
dropped_months = 3
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1797   0.1394   0.0126   0.5818   0.3127   0.5707    12080
        climo_mean   0.1067   0.0803  -0.0000   0.8496   0.1079   0.8478    12080
      climo_median   0.1098   0.0777  -0.0006   0.8639   0.0970   0.8395    12080
             conor   0.1030   0.0777   0.0002   0.8670   0.0957   0.8583    12080
     interp_clim_a   0.1234   0.0955   0.0063   0.7157   0.2103   0.8075    12080
     interp_clim_b   0.1261   0.0971   0.0080   0.7088   0.2169   0.7985    12080
     interp_clim_c   0.1232   0.0946   0.0060   0.7229   0.2048   0.8062    12080
       whit_a_0p50   0.1704   0.1301   0.0133   0.6926   0.2338   0.6221    12080
       whit_a_0p20   0.1703   0.1295   0.0124   0.7323   0.2045   0.6307    12080
       whit_a_0p10   0.1724   0.1309   0.0118   0.7498   0.1913   0.6280    12080
       whit_a_0p05   0.1750   0.1328   0.0113   0.

## Randomly drop a three month gap during the growing season (Apr-Sept)

In [18]:
dropped_months = 3
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1635   0.1278  -0.0585   0.6542   0.2323   0.6158    12080
        climo_mean   0.0938   0.0715   0.0013   0.8572   0.1214   0.8543    12080
      climo_median   0.0964   0.0695   0.0034   0.8681   0.1143   0.8466    12080
             conor   0.0888   0.0680   0.0013   0.8794   0.1027   0.8695    12080
     interp_clim_a   0.1094   0.0859  -0.0286   0.7557   0.1769   0.8202    12080
     interp_clim_b   0.1103   0.0864  -0.0285   0.7531   0.1791   0.8167    12080
     interp_clim_c   0.1092   0.0851  -0.0275   0.7611   0.1733   0.8192    12080
       whit_a_0p50   0.1554   0.1188  -0.0271   0.7267   0.2027   0.6276    12080
       whit_a_0p20   0.1538   0.1171  -0.0202   0.7626   0.1794   0.6388    12080
       whit_a_0p10   0.1548   0.1179  -0.0177   0.7796   0.1676   0.6388    12080
       whit_a_0p05   0.1566  

## Randomly drop a three month gap during the non-growing season (Oct-Mar)

In [19]:
dropped_months = 3
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1938   0.1510   0.0918   0.6051   0.3197   0.4795    12017
        climo_mean   0.1190   0.0906  -0.0019   0.7358   0.1505   0.7311    12017
      climo_median   0.1228   0.0875  -0.0056   0.7502   0.1385   0.7158    12017
             conor   0.1155   0.0878  -0.0025   0.7658   0.1326   0.7473    12017
     interp_clim_a   0.1362   0.1059   0.0449   0.6705   0.2351   0.6865    12017
     interp_clim_b   0.1414   0.1092   0.0490   0.6595   0.2455   0.6660    12017
     interp_clim_c   0.1361   0.1050   0.0431   0.6776   0.2291   0.6835    12017
       whit_a_0p50   0.1824   0.1409   0.0554   0.6690   0.2464   0.4911    12017
       whit_a_0p20   0.1825   0.1406   0.0438   0.6989   0.2176   0.4916    12017
       whit_a_0p10   0.1849   0.1423   0.0388   0.7136   0.2041   0.4860    12017
       whit_a_0p05   0.187

## Randomly drop a four month gap during the year

In [20]:
dropped_months = 4
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2028   0.1593   0.0098   0.5044   0.3653   0.4545    12080
        climo_mean   0.1074   0.0808  -0.0008   0.8450   0.1104   0.8453    12080
      climo_median   0.1106   0.0783  -0.0009   0.8586   0.1006   0.8364    12080
             conor   0.1043   0.0782  -0.0007   0.8601   0.0997   0.8540    12080
     interp_clim_a   0.1336   0.1045   0.0045   0.6747   0.2379   0.7731    12080
     interp_clim_b   0.1368   0.1065   0.0063   0.6660   0.2460   0.7611    12080
     interp_clim_c   0.1335   0.1037   0.0045   0.6815   0.2330   0.7714    12080
       whit_a_0p50   0.1927   0.1469   0.0106   0.6320   0.2746   0.5248    12080
       whit_a_0p20   0.1924   0.1457   0.0091   0.6763   0.2414   0.5391    12080
       whit_a_0p10   0.1947   0.1472   0.0082   0.6960   0.2263   0.5381    12080
       whit_a_0p05   0.1977   0.1492   0.0075   0.7

## Randomly drop a four month gap during the growing season (Apr-Sept)

In [21]:
dropped_months = 4
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1961   0.1544  -0.0659   0.5622   0.2911   0.5038    12080
        climo_mean   0.0957   0.0724   0.0009   0.8616   0.1137   0.8650    12080
      climo_median   0.0983   0.0701   0.0029   0.8745   0.1053   0.8581    12080
             conor   0.0914   0.0695   0.0004   0.8808   0.0976   0.8767    12080
     interp_clim_a   0.1234   0.0974  -0.0325   0.7119   0.2024   0.8008    12080
     interp_clim_b   0.1251   0.0983  -0.0320   0.7066   0.2072   0.7937    12080
     interp_clim_c   0.1227   0.0965  -0.0315   0.7183   0.1982   0.8010    12080
       whit_a_0p50   0.1860   0.1406  -0.0302   0.6639   0.2439   0.5345    12080
       whit_a_0p20   0.1847   0.1389  -0.0218   0.7037   0.2199   0.5480    12080
       whit_a_0p10   0.1863   0.1400  -0.0187   0.7219   0.2080   0.5482    12080
       whit_a_0p05   0.1887   

## Randomly drop a four month gap during the non-growing season (Oct-Mar)

In [22]:
dropped_months = 4
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2026   0.1586   0.0921   0.5719   0.3536   0.4780    12017
        climo_mean   0.1172   0.0885  -0.0020   0.7732   0.1365   0.7720    12017
      climo_median   0.1206   0.0852  -0.0051   0.7865   0.1254   0.7598    12017
             conor   0.1133   0.0856  -0.0021   0.7968   0.1220   0.7870    12017
     interp_clim_a   0.1384   0.1078   0.0450   0.6725   0.2450   0.7188    12017
     interp_clim_b   0.1434   0.1112   0.0489   0.6610   0.2560   0.7007    12017
     interp_clim_c   0.1382   0.1069   0.0435   0.6792   0.2395   0.7163    12017
       whit_a_0p50   0.1898   0.1464   0.0564   0.6638   0.2618   0.5051    12017
       whit_a_0p20   0.1905   0.1464   0.0448   0.6967   0.2301   0.5043    12017
       whit_a_0p10   0.1936   0.1487   0.0398   0.7123   0.2155   0.4971    12017
       whit_a_0p05   0.1972

## Randomly drop a six month gap during the year

In [23]:
dropped_months = 6
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2496   0.1969   0.0056   0.3854   0.4487   0.2430    12080
        climo_mean   0.1057   0.0801  -0.0008   0.8511   0.1066   0.8493    12080
      climo_median   0.1082   0.0773  -0.0010   0.8664   0.0953   0.8426    12080
             conor   0.1029   0.0780  -0.0003   0.8620   0.0992   0.8570    12080
     interp_clim_a   0.1512   0.1196   0.0024   0.6183   0.2776   0.7012    12080
     interp_clim_b   0.1555   0.1224   0.0050   0.6062   0.2889   0.6826    12080
     interp_clim_c   0.1505   0.1186   0.0023   0.6259   0.2720   0.7028    12080
       whit_a_0p50   0.2430   0.1848   0.0065   0.5173   0.3545   0.3220    12080
       whit_a_0p20   0.2411   0.1820   0.0066   0.5670   0.3188   0.3501    12080
       whit_a_0p10   0.2428   0.1832   0.0064   0.5896   0.3023   0.3566    12080
       whit_a_0p05   0.2454   0.1855   0.0061   0.60

## Randomly drop a six month gap during the growing season (Apr-Sept)

In [24]:
dropped_months = 6
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2621   0.2072  -0.0606   0.3671   0.4221   0.2255    12080
        climo_mean   0.1028   0.0776   0.0006   0.8613   0.1064   0.8597    12080
      climo_median   0.1063   0.0756   0.0007   0.8756   0.0956   0.8506    12080
             conor   0.1006   0.0758   0.0007   0.8708   0.0992   0.8657    12080
     interp_clim_a   0.1554   0.1231  -0.0300   0.6142   0.2642   0.7025    12080
     interp_clim_b   0.1588   0.1252  -0.0283   0.6034   0.2741   0.6859    12080
     interp_clim_c   0.1548   0.1222  -0.0299   0.6213   0.2589   0.7032    12080
       whit_a_0p50   0.2673   0.2011  -0.0486   0.4800   0.3480   0.2625    12080
       whit_a_0p20   0.2659   0.1989  -0.0423   0.5260   0.3192   0.2861    12080
       whit_a_0p10   0.2675   0.1999  -0.0392   0.5465   0.3067   0.2920    12080
       whit_a_0p05   0.2699   0

## Randomly drop a six month gap during the non-growing season (Oct-Mar)

In [25]:
dropped_months = 6
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue

    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2388   0.1881   0.0698   0.4449   0.4437   0.3098    12017
        climo_mean   0.1100   0.0828  -0.0000   0.8266   0.1168   0.8262    12017
      climo_median   0.1131   0.0796  -0.0012   0.8407   0.1061   0.8171    12017
             conor   0.1071   0.0805  -0.0006   0.8405   0.1069   0.8354    12017
     interp_clim_a   0.1496   0.1171   0.0349   0.6357   0.2803   0.7021    12017
     interp_clim_b   0.1548   0.1206   0.0384   0.6229   0.2925   0.6821    12017
     interp_clim_c   0.1493   0.1164   0.0343   0.6428   0.2749   0.7013    12017
       whit_a_0p50   0.2214   0.1708   0.0533   0.5886   0.3304   0.4121    12017
       whit_a_0p20   0.2202   0.1691   0.0451   0.6358   0.2904   0.4305    12017
       whit_a_0p10   0.2228   0.1709   0.0404   0.6566   0.2718   0.4299    12017
       whit_a_0p05   0.2262 

### Old plotting function code

In [26]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.2}, 'line_kws': {'color': 'red'}},
# )

In [27]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='hist')

In [28]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='kde')

In [29]:
# g = sns.pairplot(
#     output_df, x_vars = ['original'], y_vars = comparison_cols, kind='reg', 
#     plot_kws={'scatter_kws': {'s': 1, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )
# for ax in g.axes.flatten():
#     if ax:
#         ax.set_xlim(0, 1.2)
#         ax.set_ylim(0, 1.2)
#         ax.set_aspect('equal')
#         ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2])
#         ax.axline((0, 0), slope=1, color='gray', linestyle='--')  # linewidth=1
# plt.show()

In [30]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )